<a href="https://colab.research.google.com/github/BryanDYang/Optiver_Trading/blob/main/Optiver_Trading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Libabry

In [30]:
# import packages
import glob
import pandas as pd
import numpy as np
import datetime as dt
import re
import os
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import cm
from collections import Counter
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.impute import SimpleImputer

## Import Data

In [2]:
%%capture
!apt update
!pip install kaggle

In [ ]:
# Run this cell to mount your drive (you will be prompted to sign in)
from google.colab import drive
drive.mount('/content/drive')

In [4]:
# Create the kaggle directory and
# (NOTE: Do NOT run this cell more than once unless restarting kernel)
!mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [7]:
# Read the uploaded kaggle.json file
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/

In [8]:
!chmod 600 ~/.kaggle/kaggle.json

In [11]:
# Download dataset
!kaggle competitions download optiver-trading-at-the-close

 99% 199M/201M [00:01<00:00, 141MB/s]
100% 201M/201M [00:01<00:00, 164MB/s]


In [12]:
!unzip optiver-trading-at-the-close.zip

Archive:  optiver-trading-at-the-close.zip
replace example_test_files/revealed_targets.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: example_test_files/revealed_targets.csv  
replace example_test_files/sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: example_test_files/sample_submission.csv  
replace example_test_files/test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: example_test_files/test.csv  
  inflating: optiver2023/__init__.py  
replace optiver2023/competition.cpython-310-x86_64-linux-gnu.so? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: optiver2023/competition.cpython-310-x86_64-linux-gnu.so  
  inflating: public_timeseries_testing_util.py  
  inflating: train.csv               y



In [13]:
df = pd.read_csv("train.csv")
df = df.dropna(subset=["target"])
df.reset_index(drop=True, inplace=True)
df.shape

(5237892, 17)

## Exploratory Data

In [14]:
df.head()

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
0,0,0,0,3180602.69,1,0.999812,13380276.64,NaN,NaN,0.999812,60651.50,1.000026,8493.03,1.0,-3.029704,0,0_0_0
1,1,0,0,166603.91,-1,0.999896,1642214.25,NaN,NaN,0.999896,3233.04,1.000660,20605.09,1.0,-5.519986,0,0_0_1
2,2,0,0,302879.87,-1,0.999561,1819368.03,NaN,NaN,0.999403,37956.00,1.000298,18995.00,1.0,-8.389950,0,0_0_2
3,3,0,0,11917682.27,-1,1.000171,18389745.62,NaN,NaN,0.999999,2324.90,1.000214,479032.40,1.0,-4.010200,0,0_0_3
4,4,0,0,447549.96,-1,0.999532,17860614.95,NaN,NaN,0.999394,16485.54,1.000016,434.10,1.0,-7.349849,0,0_0_4


In [15]:
num_nulls = df.isnull().any(axis=1).sum()
num_nulls

2894254

In [16]:
df['far_price'].isnull().sum()

2894254

In [17]:
df['near_price'].isnull().sum()

2857092

In [18]:
num_dups = df.duplicated().sum()
num_dups

0

In [19]:
df.dtypes

stock_id                     int64
date_id                      int64
seconds_in_bucket            int64
imbalance_size             float64
imbalance_buy_sell_flag      int64
reference_price            float64
matched_size               float64
far_price                  float64
near_price                 float64
bid_price                  float64
bid_size                   float64
ask_price                  float64
ask_size                   float64
wap                        float64
target                     float64
time_id                      int64
row_id                      object
dtype: object

In [20]:
df.describe()

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id
count,5.237892e+06,5.237892e+06,5.237892e+06,5.237760e+06,5.237892e+06,5.237760e+06,5.237760e+06,2.343638e+06,2.380800e+06,5.237760e+06,5.237892e+06,5.237760e+06,5.237892e+06,5.237760e+06,5.237892e+06,5.237892e+06
mean,9.928777e+01,2.415097e+02,2.700008e+02,5.715293e+06,-1.189620e-02,9.999955e-01,4.510025e+07,1.001713e+00,9.996601e-01,9.997263e-01,5.181445e+04,1.000264e+00,5.357658e+04,9.999920e-01,-4.756125e-02,1.331003e+04
std,5.787187e+01,1.385313e+02,1.587450e+02,2.051591e+07,8.853447e-01,2.532497e-03,1.398413e+08,7.214705e-01,1.216920e-02,2.499345e-03,1.114221e+05,2.510042e-03,1.293563e+05,2.497509e-03,9.452860e+00,7.619238e+03
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,9.352850e-01,4.316610e+03,7.700000e-05,7.869880e-01,9.349150e-01,5.600000e-01,9.398270e-01,5.900000e-01,9.380080e-01,-3.852898e+02,0.000000e+00
25%,4.900000e+01,1.220000e+02,1.300000e+02,8.453415e+04,-1.000000e+00,9.987630e-01,5.279575e+06,9.963320e-01,9.971000e-01,9.985290e-01,7.375160e+03,9.990290e-01,7.824000e+03,9.987810e-01,-4.559755e+00,6.729000e+03
50%,9.900000e+01,2.420000e+02,2.700000e+02,1.113604e+06,0.000000e+00,9.999670e-01,1.288264e+07,9.998830e-01,9.998890e-01,9.997280e-01,2.196980e+04,1.000207e+00,2.301867e+04,9.999970e-01,-6.020069e-02,1.334500e+04
75%,1.490000e+02,3.610000e+02,4.100000e+02,4.190951e+06,1.000000e+00,1.001174e+00,3.270013e+07,1.003318e+00,1.002590e+00,1.000905e+00,5.583240e+04,1.001414e+00,5.787932e+04,1.001149e+00,4.409552e+00,1.990700e+04
max,1.990000e+02,4.800000e+02,5.400000e+02,2.982028e+09,1.000000e+00,1.077488e+00,7.713682e+09,4.379531e+02,1.309732e+00,1.077488e+00,3.028784e+07,1.077836e+00,5.440500e+07,1.077675e+00,4.460704e+02,2.645400e+04


In [21]:
df['stock_id'].nunique()

200

In [22]:
df['date_id'].nunique()

481

## Modeling

### Linear Regression

In [79]:
# store features dataframe into "features" variable
features = df.drop(columns='target')

# store the classification target variable into "target"
target = df['target']

In [80]:
# assign appropriate value to seed and conduct 80/20 train-test split with random_state = seed
Seed = 5

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=Seed)

In [81]:
imputer = SimpleImputer(strategy='most_frequent') # mean, median, or most_frequent
X_train = imputer.fit_transform(X_train)
X_test = imputer.fit_transform(X_test)

In [82]:
#Initialize model with default parameters and fit it on the training set
model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)

mae

6.31642461971782

### Linear Regression in Time-Series

In [86]:
# TimeSeries Split for cross-validation
tscv = TimeSeriesSplit(n_splits=10)
mae_scores = []

for train_index, test_index in tscv.split(features):

    X_train, X_test = features.iloc[train_index], features.iloc[test_index]
    y_train, y_test = target.iloc[train_index], target.iloc[test_index]

    X_train = imputer.fit_transform(X_train)
    X_test = imputer.fit_transform(X_test)

    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    mae_scores.append(mae)

avg_mae = np.mean(mae_scores)

avg_mae

6.432166811883237

### Hyperparameter Tuning